# Setup camera calibration sphere

## Find tag crop positions

In [ ]:
from time import sleep
import sys

sys.stderr = sys.__stderr__  # redirect stderr back to the shell

from pupil_apriltags import Detector
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibration_helper.trajectories.camera_sphere import CameraCalibrationSphere, make_im_slice
from skrgbd.devices.rig import Rig
from skrgbd.devices.robot.robot_on_sphere import RobotOnSphere, RobotOnSTLSphere

In [ ]:
RobotOnSTLSphere().move_to((.5, .5), .1);

Включите ScanCenter, включите проекцию Cross на проекторе и расположите калибровочную доску так чтобы
* тег был над меткой на столе,
* тег был в центре правой СТЛ камеры,
* доска была параллельна ригу.

In [ ]:
robot = RobotOnSphere()
trajectory = CameraCalibrationSphere(robot)
trajectory.move_zero(.1)

In [ ]:
rig = Rig()
rig.init_cameras().join()
rig.start_cameras().join()

In [ ]:
detector = Detector(
    families='tag36h11',
    nthreads=1,
    quad_decimate=1.0,
    quad_sigma=0.0,
    refine_edges=1,
    decode_sharpening=0.25,
    debug=0
)

In [ ]:
# image = rig.realsense.snap_frame()['image']
# image = rig.tis_left.snap_frame()['image']
# image = rig.tis_right.snap_frame()['image']
# image = rig.kinect.snap_frame()['image']
# image = rig.phone_left.get_photo()
image = rig.phone_right.get_photo()

if image.ndim > 2:
    image = image[..., 1]  # keep only green channel since tag detector only works with single-channel images
# plt.imshow(image, cmap='gray')
plt.imshow(image[::8, ::8], cmap='gray')

In [ ]:
detection = detector.detect(image)[0]

x, y = detection.center
size = (detection.corners[0, 0] - detection.center[0]) * 2

size = size * 1.3
size = round(size)
x = round(x)
y = round(y)

print(f'({y}, {x}), {size}')
plt.imshow(image[make_im_slice([y, x], size)], cmap='gray')

## Check tag crop positions

In [ ]:
from time import sleep
import sys

sys.stderr = sys.__stderr__  # redirect stderr back to the shell

sys.path.append('/home/universal/Downloads/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.calibration_helper.trajectories.camera_sphere import CameraCalibrationSphere
from skrgbd.devices.rig import Rig

In [ ]:
rig = Rig()
rig.init_cameras().join()
rig.start_cameras().join()

In [ ]:
trajectory = CameraCalibrationSphere(None)
trajectory.stream_tag(rig.realsense, rig.tis_left, rig.tis_right, rig.kinect, rig.phone_left, rig.phone_right)

In [ ]:
trajectory.stop_tag_streaming()